In [1]:
from tfga import GeometricAlgebra
from tfga.blades import BladeKind
import tensorflow as tf
import numpy as np
from itertools import combinations_with_replacement
import os

In [2]:
pga = GeometricAlgebra([0, 1, 1])

In [3]:
def generate_multiplication_tables(cayley):
    num_blades = cayley.shape[0]
    mult_table = np.empty([num_blades, num_blades], dtype=np.int64)
    mult_sign_table = np.zeros([num_blades, num_blades])
    mult_table.fill(-1)

    for row, row_vals in enumerate(cayley):
        for col, col_vals in enumerate(row_vals):
            if tf.reduce_any(col_vals != 0):
                mult_table[row, col] = tf.argmax(tf.abs(col_vals))
                mult_sign_table[row, col] = cayley[row, col, mult_table[row, col]]
    return mult_table, mult_sign_table

mult_table, mult_sign_table = generate_multiplication_tables(pga.cayley)
mult_table_inner, mult_sign_table_inner = generate_multiplication_tables(pga.cayley_inner)
mult_table_outer, mult_sign_table_outer = generate_multiplication_tables(pga.cayley_outer)

print(mult_table)
print(mult_sign_table)

[[ 0  1  2  3  4  5  6  7]
 [ 1 -1  4  5 -1 -1  7 -1]
 [ 2  4  0  6  1  7  3  5]
 [ 3  5  6  0  7  1  2  4]
 [ 4 -1  1  7 -1 -1  5 -1]
 [ 5 -1  7  1 -1 -1  4 -1]
 [ 6  7  3  2  5  4  0  1]
 [ 7 -1  5  4 -1 -1  1 -1]]
[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  0.  1.  1.  0.  0.  1.  0.]
 [ 1. -1.  1.  1. -1. -1.  1. -1.]
 [ 1. -1. -1.  1.  1. -1. -1.  1.]
 [ 1.  0.  1.  1.  0.  0.  1.  0.]
 [ 1.  0. -1.  1.  0.  0. -1.  0.]
 [ 1.  1. -1.  1. -1.  1. -1. -1.]
 [ 1.  0. -1.  1.  0.  0. -1.  0.]]


In [4]:
def blade_repr(blade):
    return "Scalar" if blade == "" else "E%s" % blade

blade_reprs = list(map(blade_repr, pga.blades))

def generate_blade_types(blade_reprs):
    type_lines = []
    for blade in blade_reprs:
        type_lines.append("export type Blade%s = { %s: number }" % (blade, blade.lower()))
    return "\n".join(type_lines)

def generate_optional_mv(blade_reprs):
    type_lines = ["export type OptionalMultiVector = {"]
    for blade in blade_reprs:
        type_lines.append("    %s?: number" % blade.lower())
    type_lines.append("}")
    return "\n".join(type_lines)

print(generate_blade_types(blade_reprs))
print(generate_optional_mv(blade_reprs))

export type BladeScalar = { scalar: number }
export type BladeE0 = { e0: number }
export type BladeE1 = { e1: number }
export type BladeE2 = { e2: number }
export type BladeE01 = { e01: number }
export type BladeE02 = { e02: number }
export type BladeE12 = { e12: number }
export type BladeE012 = { e012: number }
export type OptionalMultiVector = {
    scalar?: number
    e0?: number
    e1?: number
    e2?: number
    e01?: number
    e02?: number
    e12?: number
    e012?: number
}


In [5]:
def generate_kind_types(kinds, ga, blade_reprs):
    type_lines = []
    for kind, kind_name in kinds:
        kind_indices = ga.get_kind_blade_indices(kind)
        if len(kind_indices) > 0:
            kind_blades = " & ".join(map(lambda blade_index: "Blade%s" % blade_reprs[blade_index], kind_indices))
            type_lines.append("export type %s = %s" % (kind_name, kind_blades))
    return "\n".join(type_lines)

kinds = [
    (BladeKind.SCALAR, "Scalar"),
    (BladeKind.VECTOR, "Vector"),
    (BladeKind.BIVECTOR, "BiVector"),
    (BladeKind.TRIVECTOR, "TriVector"),
    (BladeKind.PSEUDOTRIVECTOR, "PseudoTriVector"),
    (BladeKind.PSEUDOBIVECTOR, "PseudoBiVector"),
    (BladeKind.PSEUDOVECTOR, "PseudoVector"),
    (BladeKind.PSEUDOSCALAR, "PseudoScalar"),
    (BladeKind.EVEN, "Even"),
    (BladeKind.ODD, "Odd"),
    (BladeKind.MV, "MultiVector")
]

print(generate_kind_types(kinds, pga, blade_reprs))

export type Scalar = BladeScalar
export type Vector = BladeE0 & BladeE1 & BladeE2
export type BiVector = BladeE01 & BladeE02 & BladeE12
export type TriVector = BladeE012
export type PseudoTriVector = BladeScalar
export type PseudoBiVector = BladeE0 & BladeE1 & BladeE2
export type PseudoVector = BladeE01 & BladeE02 & BladeE12
export type PseudoScalar = BladeE012
export type Even = BladeScalar & BladeE01 & BladeE02 & BladeE12
export type Odd = BladeE0 & BladeE1 & BladeE2 & BladeE012
export type MultiVector = BladeScalar & BladeE0 & BladeE1 & BladeE2 & BladeE01 & BladeE02 & BladeE12 & BladeE012


In [6]:
def generate_add_type(blade_reprs):
    type_lines = ["export type AddResultType<A, B> = ("]
    for i, blade in enumerate(blade_reprs):
        type_lines.append("    (A extends Blade%s ? Blade%s : {}) &" % (blade, blade))
        type_lines.append("    (B extends Blade%s ? Blade%s : {})%s" % (blade, blade, "" if i == len(blade_reprs) - 1 else " &"))
    type_lines.append(")")
    return "\n".join(type_lines)

def generate_add(name, op, blade_reprs):
    lines = [
        "export const %s = <A extends OptionalMultiVector, B extends OptionalMultiVector>(a: A, b: B): AddResultType<A, B> => {" % name,
        "    const result: any = {"
    ]
    for blade in blade_reprs:
        lines.append("        %s: (a.%s !== undefined || b.%s !== undefined) ? (a.%s || 0) %s (b.%s || 0) : undefined," % (blade.lower(), blade.lower(), blade.lower(), blade.lower(), op, blade.lower()))
    lines += [
        "    }",
        "    return result as AddResultType<A, B>",
        "}"
    ]
    return "\n".join(lines)

print(generate_add_type(blade_reprs))
print(generate_add("Add", "+", blade_reprs))
print(generate_add("Sub", "-", blade_reprs))

export type AddResultType<A, B> = (
    (A extends BladeScalar ? BladeScalar : {}) &
    (B extends BladeScalar ? BladeScalar : {}) &
    (A extends BladeE0 ? BladeE0 : {}) &
    (B extends BladeE0 ? BladeE0 : {}) &
    (A extends BladeE1 ? BladeE1 : {}) &
    (B extends BladeE1 ? BladeE1 : {}) &
    (A extends BladeE2 ? BladeE2 : {}) &
    (B extends BladeE2 ? BladeE2 : {}) &
    (A extends BladeE01 ? BladeE01 : {}) &
    (B extends BladeE01 ? BladeE01 : {}) &
    (A extends BladeE02 ? BladeE02 : {}) &
    (B extends BladeE02 ? BladeE02 : {}) &
    (A extends BladeE12 ? BladeE12 : {}) &
    (B extends BladeE12 ? BladeE12 : {}) &
    (A extends BladeE012 ? BladeE012 : {}) &
    (B extends BladeE012 ? BladeE012 : {})
)
export const Add = <A extends OptionalMultiVector, B extends OptionalMultiVector>(a: A, b: B): AddResultType<A, B> => {
    const result: any = {
        scalar: (a.scalar !== undefined || b.scalar !== undefined) ? (a.scalar || 0) + (b.scalar || 0) : undefined,
        e0

In [7]:
def generate_dual_type(blade_reprs, dual_blade_indices):
    type_lines = ["export type DualResultType<A> = ("]
    for i, blade in enumerate(blade_reprs):
        dual_blade_index = dual_blade_indices[i]
        dual_blade = blade_reprs[dual_blade_index]
        type_lines.append("    (A extends Blade%s ? Blade%s : {})%s" % (blade, dual_blade, "" if i == len(blade_reprs) - 1 else " &"))
    type_lines.append(")")
    return "\n".join(type_lines)

def generate_dual(blade_reprs, dual_blade_indices, dual_blade_signs):
    lines = [
        "export const dual = <A extends OptionalMultiVector>(a: A): DualResultType<A> => {",
        "    const result: any = {"
    ]

    for i, blade in enumerate(blade_reprs):
        dual_blade_index = dual_blade_indices[i]
        dual_blade_sign = dual_blade_signs[i]
        dual_blade = blade_reprs[dual_blade_index]
        lines.append("        %s: a.%s !== undefined ? %s * a.%s : undefined," % (blade.lower(), dual_blade.lower(), dual_blade_sign.numpy(), dual_blade.lower()))

    lines += [
        "    }",
        "    return result as DualResultType<A>",
        "}"
    ]
    return "\n".join(lines)

print(generate_dual_type(blade_reprs, pga.dual_blade_indices))
print(generate_dual(blade_reprs, pga.dual_blade_indices, pga.dual_blade_signs))

export type DualResultType<A> = (
    (A extends BladeScalar ? BladeE012 : {}) &
    (A extends BladeE0 ? BladeE12 : {}) &
    (A extends BladeE1 ? BladeE02 : {}) &
    (A extends BladeE2 ? BladeE01 : {}) &
    (A extends BladeE01 ? BladeE2 : {}) &
    (A extends BladeE02 ? BladeE1 : {}) &
    (A extends BladeE12 ? BladeE0 : {}) &
    (A extends BladeE012 ? BladeScalar : {})
)
export const dual = <A extends OptionalMultiVector>(a: A): DualResultType<A> => {
    const result: any = {
        scalar: a.e012 !== undefined ? 1.0 * a.e012 : undefined,
        e0: a.e12 !== undefined ? 1.0 * a.e12 : undefined,
        e1: a.e02 !== undefined ? -1.0 * a.e02 : undefined,
        e2: a.e01 !== undefined ? 1.0 * a.e01 : undefined,
        e01: a.e2 !== undefined ? 1.0 * a.e2 : undefined,
        e02: a.e1 !== undefined ? -1.0 * a.e1 : undefined,
        e12: a.e0 !== undefined ? 1.0 * a.e0 : undefined,
        e012: a.scalar !== undefined ? 1.0 * a.scalar : undefined,
    }
    return result as 

In [8]:
def generate_geom_prod_type(name, blade_reprs, mult_table):
    type_lines = []
    for i, blade_a in enumerate(blade_reprs):
        a_type_lines = []
        for j, blade_b in enumerate(blade_reprs):
            result_blade_index = mult_table[i, j]
            if result_blade_index >= 0:
                result_blade = blade_reprs[result_blade_index]

                if len(a_type_lines) > 0:
                    a_type_lines[-1] += " &"

                a_type_lines.append("        (B extends Blade%s ? Blade%s : {})" % (blade_b, result_blade))

        if len(a_type_lines) > 0:
            if len(type_lines) > 0:
                type_lines[-1] += " &"
            type_lines.append("    (A extends Blade%s ? (" % blade_a)
            type_lines += a_type_lines
            type_lines.append("    ) : {})")
    type_lines = ["export type %sResultType<A, B> =" % name] + type_lines
    return "\n".join(type_lines)

def generate_geom_prod(name, blade_reprs, mult_table, mult_sign_table):
    lines = [
        "export const %s = <A extends OptionalMultiVector, B extends OptionalMultiVector>(a: A, b: B): %sResultType<A, B> => {" % (name[0].lower() + name[1:], name)
    ]
    for i, result_blade in enumerate(blade_reprs):
        lines.append("    let result%s = undefined" % result_blade)
        result_blade_factors = np.where(mult_table == i)
        
        if result_blade_factors[0].shape[0] > 0:
            assignment_lines = []
            bool_names = []
            for j, (blade_ind_a, blade_ind_b) in enumerate(zip(*result_blade_factors)):
                blade_a = blade_reprs[blade_ind_a]
                blade_b = blade_reprs[blade_ind_b]
                prod_sign = mult_sign_table[blade_ind_a, blade_ind_b]

                bool_name = "%s_%d" % (result_blade.lower(), j)
                bool_names.append(bool_name)

                lines.append("    const %s = a.%s !== undefined && b.%s !== undefined" % (bool_name, blade_a.lower(), blade_b.lower()))
                assignment_lines.append("        if (%s) result%s += %s * (a.%s! * b.%s!)" % (bool_name, result_blade, prod_sign, blade_a.lower(), blade_b.lower()))
            lines += [
                "    if (%s) {" % " || ".join(bool_names),
                "        result%s = 0" % result_blade
            ]
            lines += assignment_lines
            lines.append("    }")

    lines.append("    const result: any = {")

    for i, result_blade in enumerate(blade_reprs):
        lines.append("        %s: result%s," % (result_blade.lower(), result_blade))

    lines += [
        "    }",
        "    return result as %sResultType<A, B>" % name,
        "}"
    ]
    return "\n".join(lines)

print(generate_geom_prod_type("GeometricProduct", blade_reprs, mult_table))
print(generate_geom_prod("GeometricProduct", blade_reprs, mult_table, mult_sign_table))

export type GeometricProductResultType<A, B> =
    (A extends BladeScalar ? (
        (B extends BladeScalar ? BladeScalar : {}) &
        (B extends BladeE0 ? BladeE0 : {}) &
        (B extends BladeE1 ? BladeE1 : {}) &
        (B extends BladeE2 ? BladeE2 : {}) &
        (B extends BladeE01 ? BladeE01 : {}) &
        (B extends BladeE02 ? BladeE02 : {}) &
        (B extends BladeE12 ? BladeE12 : {}) &
        (B extends BladeE012 ? BladeE012 : {})
    ) : {}) &
    (A extends BladeE0 ? (
        (B extends BladeScalar ? BladeE0 : {}) &
        (B extends BladeE1 ? BladeE01 : {}) &
        (B extends BladeE2 ? BladeE02 : {}) &
        (B extends BladeE12 ? BladeE012 : {})
    ) : {}) &
    (A extends BladeE1 ? (
        (B extends BladeScalar ? BladeE1 : {}) &
        (B extends BladeE0 ? BladeE01 : {}) &
        (B extends BladeE1 ? BladeScalar : {}) &
        (B extends BladeE2 ? BladeE12 : {}) &
        (B extends BladeE01 ? BladeE0 : {}) &
        (B extends BladeE02 ? BladeE012 :

In [9]:
def generate_elementwise(name, op, blade_reprs):
    lines = [
        "export const %s = <A extends OptionalMultiVector>(a: A, b: number): A => {" % name,
        "    const result: any = {"
    ]

    for blade in blade_reprs:
        lines.append("        %s: a.%s !== undefined ? a.%s %s b : undefined," % (blade.lower(), blade.lower(), blade.lower(), op))

    lines += [
        "    }",
        "    return result as A",
        "}"
    ]

    return "\n".join(lines)

print(generate_elementwise("multiply", "*", blade_reprs))

export const multiply = <A extends OptionalMultiVector>(a: A, b: number): A => {
    const result: any = {
        scalar: a.scalar !== undefined ? a.scalar * b : undefined,
        e0: a.e0 !== undefined ? a.e0 * b : undefined,
        e1: a.e1 !== undefined ? a.e1 * b : undefined,
        e2: a.e2 !== undefined ? a.e2 * b : undefined,
        e01: a.e01 !== undefined ? a.e01 * b : undefined,
        e02: a.e02 !== undefined ? a.e02 * b : undefined,
        e12: a.e12 !== undefined ? a.e12 * b : undefined,
        e012: a.e012 !== undefined ? a.e012 * b : undefined,
    }
    return result as A
}


In [10]:
def generate_reversion(blade_degrees, blade_reprs):
    lines = [
        "export const reversion = <A extends OptionalMultiVector>(a: A): A => {",
        "    const result: any = {"
    ]
    blade_degrees = tf.cast(blade_degrees, tf.float32)
    odd_swaps = tf.cast(tf.floor(
        blade_degrees * (blade_degrees - 0.5)
    ) % 2, tf.float32)

    # [0, 1] -> [-1, 1]
    rev_signs = 1.0 - 2.0 * odd_swaps
    for rev_sign, blade_degree, blade in zip(rev_signs, blade_degrees, blade_reprs):
        if rev_sign > 0:
            lines.append("        %s: a.%s && a.%s," % (blade.lower(), blade.lower(), blade.lower()))
        else:
            lines.append("        %s: a.%s && -a.%s," % (blade.lower(), blade.lower(), blade.lower()))

    lines += [
        "    }",
        "    return result as A",
        "}"
    ]
    return "\n".join(lines)

print(generate_reversion(pga.blade_degrees, blade_reprs))

export const reversion = <A extends OptionalMultiVector>(a: A): A => {
    const result: any = {
        scalar: a.scalar && a.scalar,
        e0: a.e0 && a.e0,
        e1: a.e1 && a.e1,
        e2: a.e2 && a.e2,
        e01: a.e01 && -a.e01,
        e02: a.e02 && -a.e02,
        e12: a.e12 && -a.e12,
        e012: a.e012 && -a.e012,
    }
    return result as A
}


In [11]:
def generate_repr(blade_reprs):
    lines = [
        "export const repr = <A extends OptionalMultiVector>(a: A, digits: number = 3): string => {",
        "    let result = \"\""
    ]

    for blade in blade_reprs:
        b = "" if blade == "Scalar" else blade.lower()
        lines += [
            "    if (a.%s !== undefined) {" % blade.lower(),
            "        if (result === \"\") {",
            "            result += a.%s.toFixed(digits) + \"%s\"" % (
                blade.lower(), b
            ),
            "        } else {",
            "            result += a.%s >= 0 ? \" + \" + a.%s.toFixed(digits) + \"%s\" : \" - \" + Math.abs(a.%s).toFixed(digits) + \"%s\"" % (
                blade.lower(), blade.lower(), b, blade.lower(), b
            ),
            "        }",
            "    }"
        ]

    lines += [
        "    return result",
        "}"
    ]
    return "\n".join(lines)

print(generate_repr(blade_reprs))

export const repr = <A extends OptionalMultiVector>(a: A, digits: number = 3): string => {
    let result = ""
    if (a.scalar !== undefined) {
        if (result === "") {
            result += a.scalar.toFixed(digits) + ""
        } else {
            result += a.scalar >= 0 ? " + " + a.scalar.toFixed(digits) + "" : " - " + Math.abs(a.scalar).toFixed(digits) + ""
        }
    }
    if (a.e0 !== undefined) {
        if (result === "") {
            result += a.e0.toFixed(digits) + "e0"
        } else {
            result += a.e0 >= 0 ? " + " + a.e0.toFixed(digits) + "e0" : " - " + Math.abs(a.e0).toFixed(digits) + "e0"
        }
    }
    if (a.e1 !== undefined) {
        if (result === "") {
            result += a.e1.toFixed(digits) + "e1"
        } else {
            result += a.e1 >= 0 ? " + " + a.e1.toFixed(digits) + "e1" : " - " + Math.abs(a.e1).toFixed(digits) + "e1"
        }
    }
    if (a.e2 !== undefined) {
        if (result === "") {
            result += a.e2.toFixed(d

In [12]:
misc_functions = """
export const regressiveProduct = <A extends OptionalMultiVector, B extends OptionalMultiVector>(a: A, b: B) =>
    dual(exteriorProduct(dual(a), dual(b)))

export const sandwichProduct = <A extends OptionalMultiVector, B extends OptionalMultiVector>(a: A, b: B) =>
    geometricProduct(b, geometricProduct(a, reversion(b)))

export const commutatorProduct = <A extends OptionalMultiVector, B extends OptionalMultiVector>(a: A, b: B) =>
    multiply(sub(geometricProduct(a, b), geometricProduct(b, a)), 0.5)

export const exponential = <A extends OptionalMultiVector>(a: A) => {
    const gp = geometricProduct(a, a) as any
    if (gp.scalar === undefined) {
        throw new Error("Input to exponential needs to square to scalar")
    }
    const s = gp.scalar as number

    if (s < -0.1) {
        const rootS = Math.sign(s) * Math.sqrt(Math.abs(s))
        return add({ scalar: Math.cos(rootS) }, multiply(a, Math.sin(rootS) / rootS))
    } else if (s > 0.1) {
        const rootS = Math.sign(s) * Math.sqrt(Math.abs(s))
        return add({ scalar: Math.cosh(rootS) }, multiply(a, Math.sinh(rootS) / rootS))
    } else {
        return add({ scalar: 1 }, a)
    }
}
""".strip()

In [13]:
def write_ts(out_path, signature):
    ga = GeometricAlgebra(signature)
    blade_reprs = list(map(blade_repr, ga.blades))
    mult_table, mult_sign_table = generate_multiplication_tables(ga.cayley)
    mult_table_inner, mult_sign_table_inner = generate_multiplication_tables(ga.cayley_inner)
    mult_table_outer, mult_sign_table_outer = generate_multiplication_tables(ga.cayley_outer)
    kinds = [
        (BladeKind.SCALAR, "Scalar"),
        (BladeKind.VECTOR, "Vector"),
        (BladeKind.BIVECTOR, "BiVector"),
        (BladeKind.TRIVECTOR, "TriVector"),
        (BladeKind.PSEUDOTRIVECTOR, "PseudoTriVector"),
        (BladeKind.PSEUDOBIVECTOR, "PseudoBiVector"),
        (BladeKind.PSEUDOVECTOR, "PseudoVector"),
        (BladeKind.PSEUDOSCALAR, "PseudoScalar"),
        (BladeKind.EVEN, "Even"),
        (BladeKind.ODD, "Odd"),
        (BladeKind.MV, "MultiVector")
    ]

    out_texts = [
        generate_blade_types(blade_reprs),
        generate_kind_types(kinds, ga, blade_reprs),
        generate_optional_mv(blade_reprs),
        generate_add_type(blade_reprs),
        generate_add("add", "+", blade_reprs),
        generate_add("sub", "-", blade_reprs),
        generate_dual_type(blade_reprs, ga.dual_blade_indices),
        generate_dual(blade_reprs, ga.dual_blade_indices, ga.dual_blade_signs),
        generate_geom_prod_type("GeometricProduct", blade_reprs, mult_table),
        generate_geom_prod("GeometricProduct", blade_reprs, mult_table, mult_sign_table),
        generate_geom_prod_type("InnerProduct", blade_reprs, mult_table_inner),
        generate_geom_prod("InnerProduct", blade_reprs, mult_table_inner, mult_sign_table_inner),
        generate_geom_prod_type("ExteriorProduct", blade_reprs, mult_table_outer),
        generate_geom_prod("ExteriorProduct", blade_reprs, mult_table_outer, mult_sign_table_outer),
        generate_elementwise("multiply", "*", blade_reprs),
        generate_elementwise("div", "/", blade_reprs),
        generate_reversion(ga.blade_degrees, blade_reprs),
        generate_repr(blade_reprs),
        misc_functions
    ]

    with open(out_path, "w", encoding="utf-8") as out_file:
        out_file.write("\n\n".join(out_texts))

os.makedirs("generated", exist_ok=True)

for i in range(5):
    for signature in combinations_with_replacement([0, -1, 1], r=i):
        file_name = "ga_%s.ts" % "".join(map(lambda s: "m" if s < 0 else ("p" if s > 0 else "z"), signature)) if i > 0 else "ga_s.ts"
        write_ts(os.path.join("generated", file_name), signature)